#### But du modèle: 

##### Prédire et prévenir les blessures causées lors de matchs de tennis. 

# **Importation des différents modules**

In [3]:
import re

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import requests
from io import BytesIO
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

# **Étape 1 : Préparation & Nettoyage des données**

### **1. Chargement du DataSet**

In [6]:
excel_file_path = '../DataSets/BlessuresTennis_Causes.xlsx'
df = pd.read_excel(excel_file_path)
df.head()

,ID,CategBlessure,TypeBlessure,Blessure,Symptomes_Blessures,Traitement_Blessures,Prescriptions_Blessures,Prix_Prescriptions (en € ),Images_Prescriptions,Descriptifs_Prescriptions,LienAchat_Prescriptions,Causes_Blessure,Zone_Blessures,Description_Zone
0,1,Blessures aux membres supérieurs,Douleur à la main et au poignet,ténosynovite de Quervain,Inflammation douloureuse des tendons près du p...,infiltrations locales de cortisone; réeduction...,Orthèse poignet-pouce tendinite NUIT; Orthèse ...,"55,31 ; 25,99 ; 81,30",https://epitact.twic.pics/prod/2024-02/mea-que...,Soulage les douleurs liées à la tendinite De Q...,https://epitact.fr/quervimmo-orthese-immobilis...,Utilisation excessive du poignet et du pouce; ...,Zone de transition,La zone intermédiaire entre le fond de court e...
1,2,Blessures aux membres supérieurs,Douleur à la main et au poignet,ténosynovite de l’extenseur ulnaire du carpe,Inflammation douloureuse des tendons près du c...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,"23,99 ; 43,30 ; 67,29",https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Surutilisation du poignet; Mauvaise posture; M...,Zone médiane gauche; Zone médiane droite,Les zones près du centre du terrain où les jou...
2,3,Blessures aux membres supérieurs,Douleur à la main et au poignet,entorse du ligament carpien,Blessure traumatique au ligament du poignet; c...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,"23,99 ; 43,30 ; 67,29",https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Torsion excessive du poignet; Chute/Impact direct,Coins inférieur gauche; Coins supérieur gauche,Les coins du fond de court près du filet où le...
3,4,Blessures aux membres supérieurs,Douleur à la main et au poignet,fractures de fatigue (scaphoïde; os crochu ou ...,Fissures dans les os du poignet ou des doigts ...,NaN,NaN,NaN,NaN,NaN,NaN,Stress répété; Impact direct; Surutilisation,Zone de service déuce,La zone près du filet du côté déuce où les jou...
4,5,Blessures aux membres supérieurs,Douleur à la main et au poignet,lésions du ligament annulaire du carpe,Blessures au ligament du poignet; causant doul...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,"23,99 ; 43,30 ; 67,29",https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Utilisation excessive; Mouvements répétitifs,Coins inférieur gauche; Coins supérieur gauche,Les coins du fond de court près du filet où le...
